# B. Sim Dashboard

### Imports

In [1]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports-WIP.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

In [2]:
# Schedule
pause_code(start_time='2024-06-06T12:25:00', timezone='EST')

Time until 12:25PM. -190 hours, 50 minutes, and 43 seconds.


### Games

Select start and end date (should be todaysdate for both if running day-of)

In [3]:
# start_date = "20240420"
start_date = todaysdate
end_date = todaysdate

Read in games

In [4]:
game_df = read_and_save_games(team_map, generate=True)
game_df = game_df[(game_df['date'] >= start_date) & (game_df['date'] <= end_date)].reset_index(drop=True)

### Settings

In [5]:
# Run historic matchups
historic = False
# Run matchups necessary for slate
slate_only = True

# Run:
Matchups = True
Simulations = True
Optimizer = True

In [6]:
%%time
if historic == False:
    %run "A. Night Dashboard-WIP.ipynb"

Running Night Dashboard
CPU times: total: 15.6 ms
Wall time: 49.1 s
CPU times: total: 15.6 ms
Wall time: 1min 2s
CPU times: total: 0 ns
Wall time: 3.51 s
20240614
STL@CHC uses Swish Analytics weather.
NYY@BOS uses Swish Analytics weather.
MIA@WSN uses Swish Analytics weather.
PHI@BAL uses Swish Analytics weather.
CLE@TOR uses Swish Analytics weather.
SDP@NYM uses Swish Analytics weather.
TBR@ATL uses Swish Analytics weather.
DET@HOU uses Swish Analytics weather.
OAK@MIN uses Swish Analytics weather.
CIN@MIL uses Swish Analytics weather.
PIT@COL uses Swish Analytics weather.
CHW@ARI uses Swish Analytics weather.
KCR@LAD uses Swish Analytics weather.
TEX@SEA uses Swish Analytics weather.
LAA@SFG uses Swish Analytics weather.
Scraping successful!
CPU times: total: 9.97 s
Wall time: 2min 28s


### Contest

In [7]:
# Read in subset of contests for which we'll create guides
subset_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '7. Subsets', f'Subset {todaysdate}.csv'))
subset_df['contestDate'] = pd.to_datetime(subset_df['contestDate'])
four_seamer_df = subset_df[(subset_df['Name'].str.contains('eamer')) & ~(subset_df['Name'].str.contains('Turbo')) & ~(subset_df['Name'].str.contains('@'))].sort_values('contestDate', ascending=True).reset_index(drop=True)
four_seamer_df

,Name,Cash Prize,Entry Fee,contestKey,draftGroupId,contestDate,contestTime,date,date_dash
0,MLB $15K Four-Seamer [20 Entry Max],"$15,000",4.0,162631008,108390,2024-06-14 19:05:00,Fri 7:05PM,20240614,NaN
1,MLB $3K Four-Seamer (Night),3000.0,4.0,162631091,108458,2024-06-14 21:40:00,Fri 9:40PM,20240614,2024-06-14


In [8]:
# Set contestKey
contestKey = four_seamer_df['contestKey'][0]

# Read in Contest Guide
guide = pd.read_csv(os.path.join(baseball_path, "A09. Contest Guides", f"Contest Guide {contestKey}.csv"))

# Identify draftGroupId
draftGroupId = guide['draftGroupId'][0]

# Identify RotoWire slate
roto_slate = guide['roto_slate'][0]

# Print out name
print(f"Date: {guide['date'][0]}. Slate: {guide['slate'][0]}. {guide['name'][0]}. {roto_slate}.")

Date: 20240614. Slate: All. MLB $15K Four-Seamer [20 Entry Max]. 16320.


In [9]:
if slate_only == True:
    slate_df = game_df[game_df['game_id'].isin(guide['game_id'].unique())].reset_index(drop=True)

### Weather

In [10]:
try:
    rotogrinders_df = pd.read_csv(os.path.join(baseball_path, "A06. Weather", "2. RotoGrinders", f"RotoGrinders {todaysdate}.csv"))
    red_list = list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Away']) + list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Home'])
    orange_list = list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Away']) + list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Home'])
    display_weather = rotogrinders_df[['Tag', 'Tag2', 'Away', 'Home', 'date', 'Description']]
except:
    red_list = []
    orange_list = []
    display_weather = None
    
display_weather

,Tag,Tag2,Away,Home,date,Description
0,green,NaN,STL,CHC,20240614,"Winds blowing in at 10mph, bump to SP."
1,orange,yellow,NYY,BOS,20240614,"The line between ""light but playable rain"" and ""deluge"" is a thin one in Boston tonight, and not one I can pinpoint this far out. There's risk here, that much I know."
2,yellow,NaN,MIA,WSH,20240614,"Any rain looks light here, probably light enough to play through without issues."
3,yellow,NaN,PHI,BAL,20240614,"Again there's some rain, but as of now models don't show anything truly disruptive."
4,orange,NaN,SD,NYM,20240614,"Heavy rain early, lighter rain late. Would think this is either a Late Start and Play, or a full-on PPD."
5,yellow,orange,PIT,COL,20240614,"Scattered showers and storms. They'll likely make it work, either with rain missing or with patience."
6,green,NaN,LAA,SF,20240614,"Winds blowing out at 20mph, but it is a park that minimizes wind impact."


[]

### B01. Matchups

In [11]:
%run "C:\Users\james\Documents\MLB\Code\B01. Matchups.ipynb"
%run "C:\Users\james\Documents\MLB\Code\A02. MLB API-WIP.ipynb"
%run "C:\Users\james\Documents\MLB\Code\A03. Steamer.ipynb"

##### Load
This loads in datasets needed to create matchup files

In [12]:
%%time
complete_dataset = pd.read_csv(os.path.join(baseball_path, "Stat Dataset.csv"))
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "Steamer Hitters.csv"))
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "Steamer Pitchers.csv"))

CPU times: total: 15.1 s
Wall time: 15.4 s


Create matchups

In [13]:
%%time
try:
    print(len(game_df))
    empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))
except:
    print("Trying Again")
    empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))

15


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   57.9s finished


CPU times: total: 35.8 s
Wall time: 58.3 s


### B02. Simulations

In [14]:
%run "C:\Users\james\Documents\MLB\Code\B02. Simulations-WIP.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U1. Imports-WIP.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

In [15]:
historic, slate_only

(False, True)

In [16]:
num_batches = int(os.cpu_count() / 1)
batch_size = int(63 * 1)

In [17]:
if historic == False:
    # Openers
    draftables = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), encoding='iso-8859-1')
    opener_list = list(draftables.query('Salary < 5000 and `Roster Position` == "P"')['Name'])
    print(f"Openers: {opener_list[:10]}")

    # Teams with Projected Lineups
    projected_lineups = pd.read_csv(os.path.join(baseball_path, "A05. Rosters", "3. Batting Orders Projected", f"Batting Orders Projected {todaysdate}.csv"))
    projected_teams = projected_lineups.merge(team_map[['SFBBTEAM', 'DKTEAM']], left_on=['team code'], right_on=['SFBBTEAM'], how='inner')
    projected_team_list = list(projected_teams.query('` confirmed` == "N"')['DKTEAM'].unique())
    print(f"Projected teams: {projected_team_list}")
else:
    opener_list = []
    projected_lineups = None
    projected_team_list = []

Openers: ['Eric Pardinho', 'Woo-Suk Go', 'Marc Church', 'Royber Salinas', 'Jack Kochanowicz', 'Chad Dallas', 'Cody Stashak', 'Chayce McDermott', 'David Buchanan', 'Brooks Wilson']
Projected teams: ['STL', 'CHC', 'NYY', 'BOS', 'MIA', 'WSH', 'PHI', 'BAL', 'CLE', 'TOR', 'SD', 'NYM', 'TB', 'ATL', 'CIN', 'MIL', 'OAK', 'MIN', 'DET', 'HOU', 'PIT', 'COL', 'CWS', 'ARI', 'TEX', 'SEA', 'KC', 'LAD', 'LAA', 'SF']


In [18]:
# Choose which games to run simulations for
if slate_only == True:
    # Just the slate
    run_df = slate_df.copy()
else:
    # All games
    run_df = game_df.copy()

In [19]:
%%time
# Print out games to simulate
_ = [print(f"{row['away_team']}@{row['home_team']} {pd.to_datetime(row['game_datetime']).tz_convert('US/Eastern').strftime('%Y-%m-%d %H:%M:%S')}") for index, row in run_df.head(15).iterrows()]

# Loop over dates of interest
for date in list(run_df['date'].unique()):
    print(date)
    
    # Extract year
    year = date[:4]
    # Extract matchups
    matchup_list = [os.path.splitext(f)[0] for f in os.listdir(os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}"))]
    
    # Loop over games
    daily_df = run_df.query(f'date == "{date}"').reset_index()
    print(daily_df.shape)
    for i in range(len(daily_df)):            
        # Extract info from run_df to look up proper file in matchups path
        away_team = daily_df['away_team'][i]
        home_team = daily_df['home_team'][i]
        game_id = daily_df['game_id'][i]
        # Beginning of filename in matchup folder
        lookup = f"{away_team}@{home_team} {game_id}"
        
        # Find the matchup file
        matchup = next((matchup for matchup in matchup_list if matchup.startswith(lookup)), None)
        print(matchup)
        
        # Create matchups
        matchup_path = os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}", f"{matchup}.xlsx")
        AwayBatters, HomeBatters, AwayPitchers, HomePitchers = create_matchup(matchup_path, batter_stats_scaler, batter_stats_fg_scaler, batter_imputations_model, pitcher_stats_scaler, pitcher_stats_fg_scaler, pitcher_imputations_model, projected_lineups)

        # Read in weather
        weather_df = pd.read_csv(os.path.join(baseball_path, "A06. Weather", "4. Park and Weather Factors", f"Park and Weather Factors {date}.csv"))
        weather_df = weather_df.query(f'game_id == {game_id}').reset_index(drop=True)
        
        # Create Scoreboard object
        game_template = Scoreboard(AwayBatters, HomeBatters, AwayPitchers, HomePitchers, 9)       
        
        ### Sim games
        # With batching (fast!)
        start = time.time()
        game_list = Parallel(n_jobs=num_batches, verbose=1)(delayed(sim_game_batch)(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, year, innings=9, debug=False, batch_size=batch_size) for batches in range(num_batches))
        game_list = [game for sublist in game_list for game in sublist]
        print(f"Simming {batch_size*num_batches} games took {time.time() - start} seconds.")
        
        # Without batching (slow!)
        # game_list = Parallel(n_jobs=16, verbose=0)(delayed(sim_game)(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, year, innings=9, debug=False) for sims in range(num_sims))
        
        # # Without joblib (slower!)
        # start = time.time()
        # game_list = []
        # for i in range(10):
        #     game = sim_game(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, year, innings=9, debug=True)
        #     game_list.append(game)
        # print(f"Simming {num_sims} games took {time.time() - start} seconds.")
        
        # Create object path
        game_path = os.path.join(baseball_path, "B02. Simulations", "1. Game Sims", f"Matchups {date}", f"{matchup}") 
        player_path = os.path.join(baseball_path, "B02. Simulations", "2. Player Sims", f"Matchups {date}", f"{matchup}") 
        # Clear folders
        if os.path.exists(game_path):
            shutil.rmtree(game_path)
        if os.path.exists(player_path):
            shutil.rmtree(player_path)
        # Make folders
        os.makedirs(game_path, exist_ok=True)
        os.makedirs(player_path, exist_ok=True)
        
        # Save each object in the list to a separate file
        for i, game_object in enumerate(game_list):
            # Delete unnecessary attributes from batter objects
            for batter in game_object.away_batters + game_object.home_batters:
                batter = batter.keep_selected_attributes()
            # Delete unnecessary attributes from pitcher objects
            for pitcher in game_object.away_pitchers + game_object.home_pitchers:
                pitcher = pitcher.keep_selected_attributes()
            # Delete unnecessary attributes from game objects
            game_object.keep_selected_attributes()

            # Constructing DataFrames for game, batters, and pitchers
            game_score_df = pd.DataFrame({
                "away_score": [game_object.away_score],
                "home_score": [game_object.home_score]
            })

            away_batters_df = pd.DataFrame([vars(batter) for batter in game_object.away_batters])
            away_batters_df['team'] = "away"
            home_batters_df = pd.DataFrame([vars(batter) for batter in game_object.home_batters])
            home_batters_df['team'] = "home"
            batters_df = pd.concat([away_batters_df, home_batters_df], axis=0)
            
            away_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.away_pitchers])
            away_pitchers_df['team'] = "away"
            home_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.home_pitchers])
            home_pitchers_df['team'] = "home"
            pitchers_df = pd.concat([away_pitchers_df, home_pitchers_df], axis=0)
            
            # Specify the file paths where you want to save the CSV files
            game_file_path = os.path.join(game_path, "game_{}.csv").format(i)
            batters_file_path = os.path.join(player_path, "batters_{}.csv").format(i)
            pitchers_file_path = os.path.join(player_path, "pitchers_{}.csv").format(i)

            # Saving DataFrames to CSV files
            game_score_df.to_csv(game_file_path, index=False)
            batters_df.to_csv(batters_file_path, index=False)
            pitchers_df.to_csv(pitchers_file_path, index=False)

        print(f'Saved to {game_path}')

PHI@BAL 2024-06-14 19:05:00
CLE@TOR 2024-06-14 19:07:00
SDP@NYM 2024-06-14 19:10:00
TBR@ATL 2024-06-14 19:20:00
DET@HOU 2024-06-14 20:10:00
OAK@MIN 2024-06-14 20:10:00
CIN@MIL 2024-06-14 20:10:00
PIT@COL 2024-06-14 20:40:00
CHW@ARI 2024-06-14 21:40:00
KCR@LAD 2024-06-14 22:10:00
TEX@SEA 2024-06-14 22:10:00
LAA@SFG 2024-06-14 22:15:00
20240614
(12, 33)
PHI@BAL 747025 1905
Away batters use Baseball Monster projected lineups
Home batters use Baseball Monster projected lineups


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   49.2s remaining:  5.8min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   52.2s finished


Simming 1008 games took 52.34707951545715 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240614\PHI@BAL 747025 1905
CLE@TOR 744924 1907
Away batters use Baseball Monster projected lineups
Home batters use Baseball Monster projected lineups


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   47.6s remaining:  5.6min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   50.2s finished


Simming 1008 games took 50.37201261520386 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240614\CLE@TOR 744924 1907
SDP@NYM 745810 1910
Away batters use Baseball Monster projected lineups
Home batters use Baseball Monster projected lineups


...

Simming 1008 games took 51.19165062904358 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240614\KCR@LAD 746134 2210
TEX@SEA 745247 2210
Away batters use Baseball Monster projected lineups
Home batters use Baseball Monster projected lineups


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   47.9s remaining:  5.6min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   50.9s finished


Simming 1008 games took 51.04162263870239 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240614\TEX@SEA 745247 2210
LAA@SFG 745326 2215
No home starter
Away batters use Baseball Monster projected lineups
Home batters use Baseball Monster projected lineups


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   46.7s remaining:  5.5min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   50.1s finished


Simming 1008 games took 50.233041524887085 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240614\LAA@SFG 745326 2215
CPU times: total: 2min 15s
Wall time: 11min 53s


In [20]:
# Optionally, read in C02, vegas scores, my scores

### B03. Lineups

In [21]:
%run "U1. Imports-WIP.ipynb"
%run "U2. Utilities.ipynb"
%run "C02. Projections.ipynb"
%run "B03. Optimizer.ipynb"

In [22]:
# Lineup ranking method
sort_by = 'Plus3'

# Set maximum ownership by position group
max_exposure_batters = 0.5
max_exposure_pitchers = 0.7

# Lineups to create
lineups = 200

# historic= False


# min_salary = 49000
# min_projection = (5, 5, 3)
# major_stack = (5, 5, 4)
# minor_stack = (2, 2, 2)
# excluded_teams = ([team for var in ['red_list', 'orange_list', 'projected_team_list'] if globals().get(var) is not None for team in globals()[var]], [team for var in ['red_list'] if globals().get(var) is not None for team in globals()[var]], [team for var in ['red_list'] if globals().get(var) is not None for team in globals()[var]])
# min_starters = (10, 10, 0)

### Run One

##### 1. Players

In [23]:
draftables_with_sims = create_player_file(contestKey, guide, draftGroupId, roto_slate, max_exposure_pitchers, max_exposure_batters)
draftables_with_sims.to_csv(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"), index=False, encoding='iso-8859-1')

In [24]:
# webbrowser.open(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"))

True

##### 2. Lineups

In [39]:
%%time
attempts = [
    ("Maximum Constraints", 49000, 5, 5, 2, red_list + orange_list, 10),
    ("Some Constraints"   , 49000, 5, 5, 2, red_list, 10),
    ("Minimum Constraints", 49000, 5, 4, 2, red_list, 0)
]


for attempt_name, min_salary, min_projection, major_stack_size, minor_stack_size, excluded_teams_list, num_confirmed_starters in attempts:
    print(f"Attempting: {attempt_name}")
    try:
        create_lineups(contestKey, min_salary, min_projection, major_stack_size, minor_stack_size, excluded_teams_list, num_confirmed_starters, lineups)
        break  # Exit loop if successful
    except Exception as e:
        print(f"Attempt failed: {e}\n")

Attempting: Maximum Constraints
Attempt failed: Can't generate lineups. Following constraints are not valid: positions_C,positions_1B,positions_SS,positions_2B_OF,positions_3B_SS,positions_2B_C_OF,positions_1B_2B_OF,positions_1B_3B_SS,positions_2B_3B_OF,positions_1B_2B_C_OF,positions_2B_3B_C_OF,positions_2B_C_OF_SS,positions_3B_C_OF_SS,positions_1B_2B_3B_OF,positions_1B_2B_3B_C_OF,positions_1B_3B_C_OF_SS,positions_2B_3B_C_OF_SS,positions_1B_2B_3B_C_OF_SS,min_starters,budget,dk_max_hitters_SEA,dk_max_hitters_TEX,dk_max_hitters_CWS,dk_max_hitters_TOR,total_players

Attempting: Some Constraints
Attempt failed: Can't generate lineups. Following constraints are not valid: positions_OF,positions_1B_OF,positions_2B_OF,positions_3B_OF,positions_1B_2B_OF,positions_1B_3B_OF,positions_2B_3B_OF,positions_2B_OF_SS,positions_3B_OF_SS,positions_1B_2B_C_OF,positions_1B_2B_3B_OF,positions_1B_2B_OF_SS,positions_1B_3B_OF_SS,positions_2B_3B_OF_SS,positions_1B_2B_C_OF_SS,positions_1B_3B_C_OF_SS,positions_1


KeyboardInterrupt



##### 3. Lineups Ranked

In [26]:
sort_by_list = ['P50', 'P75', 'P90', 'P95', 'P99', 'P100', 'Tail', 'Sim STD', 'Plus2', 'Plus3', 'rostership', 'pitcher rostership', 'batter_rostership']
lineups_ranked = choose_lineups(contestKey, roto_slate, sort_by)
lineups_ranked.to_csv(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"), index=False)

(200, 1100)


In [27]:
# webbrowser.open(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"))

In [40]:
# lineups_ranked.query('pareto == 1')

##### 4. Uploads

In [29]:
upload = create_upload_file(contestKey, sort_by)
upload.to_csv(os.path.join(baseball_path, "B03. Lineups", "4. Uploads", f"Upload {contestKey}.csv"), index=False)

##### 5. Entries

In [30]:
entry = create_entry_file(draftGroupId, contestKey)
entry.to_csv(os.path.join(baseball_path, "B03. Lineups", "5. Entries", f"Entries {draftGroupId}.csv"), index=False, encoding='iso-8859-1')

<IPython.core.display.Javascript object>

##### Upload

In [31]:
try:
    upload_entries(draftGroupId)
except:
    upload_entries(draftGroupId)

##### Email

In [32]:
email_upload_file(draftGroupId, contestKey)

### Run All

In [33]:
# Create list of contestKeys with available guides
contestKey_list = os.listdir(os.path.join(baseball_path, "A09. Contest Guides"))
contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in contestKey_list]


# Create list of contestKeys with available results
result_contestKey_list = os.listdir(os.path.join(baseball_path, "A01. DraftKings", "6. Player Results"))
result_contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in result_contestKey_list]

# Find the overlap 
available_contestKey_list = list(set(result_contestKey_list) & set(contestKey_list))

In [34]:
# %%time
# fail_list = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by, min_salary, min_projection[0], major_stack[0], minor_stack[0], excluded_teams=[], lineups, historic=True) for contestKey in available_contestKey_list)

In [35]:
# %%time
# fail_list2 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=5, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list if contestKey is not None])

In [36]:
# %%time
# fail_list3 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=4, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list2 if contestKey is not None])